# Base Concat MS MARCO

## Local folders and files

In [4]:
#Paths
collections_path = 'C:\\Users\\Matheus\\Desktop\\Pós Graduação\\NLP\\Projeto\\Datasets\\MS MARCO 2020\\collections\\collection.tsv'
qrels_train_path = 'C:\\Users\\Matheus\\Desktop\\Pós Graduação\\NLP\\Projeto\\Datasets\\MS MARCO 2020\\qrels\\qrels_train.tsv'
qrels_dev_path = 'C:\\Users\\Matheus\\Desktop\\Pós Graduação\\NLP\\Projeto\\Datasets\\MS MARCO 2020\\qrels\\qrels_dev.tsv'
queries_train_path = 'C:\\Users\\Matheus\\Desktop\\Pós Graduação\\NLP\\Projeto\\Datasets\\MS MARCO 2020\\queries\\queries_train.tsv'
queries_dev_path = 'C:\\Users\\Matheus\\Desktop\\Pós Graduação\\NLP\\Projeto\\Datasets\\MS MARCO 2020\\queries\\queries_dev.tsv'
queries_eval_path = 'C:\\Users\\Matheus\\Desktop\\Pós Graduação\\NLP\\Projeto\\Datasets\\MS MARCO 2020\\queries\\queries_eval.tsv'
#Folders
queries_topk_folder = 'C:\\Users\\Matheus\\Desktop\\Pós Graduação\\NLP\\Projeto\\Datasets\\MS MARCO 2020\\queries_topk'
chuncks_folder = 'C:\\Users\\Matheus\\Desktop\\Pós Graduação\\NLP\\Projeto\\Datasets\\MS MARCO 2020\\train_triples_small\\triples_smalls_chunks'

## Imports

In [5]:
#Bibliotecas Padrão
import os
import random
from typing import Dict
from typing import List
from typing import Tuple
import re
import gzip
import csv
import re


# Other libs
from rank_bm25 import BM25Okapi
import ftfy

#Bibliotecas Data Science
import numpy as np
import pandas as pd


## Loading files

In [6]:
def load_text_tsv(path,encoding="cp1252",file=''):
    if file != '':
        complete_path = os.path.join(path,file)
    else:
        complete_path = path
        
    csv_file =  open(complete_path, encoding=encoding,mode = 'r', errors='ignore')
    csv_reader = csv.reader(csv_file, delimiter='\t')
    rows = []
    for row in csv_reader:
        rows.append([(elem.replace("\n","")) for elem in row])
    return rows

In [11]:
def load_txts_topk(folder,k=1,n=18,encoding="cp1252"):
    k = k if k<=80 else 80
    n = n if n<=18 else 18
    nsamples = [i for i in range(0,k)] #[0] ou [0,1,2,3...]
    nparts = [i for i in range(0,n)] #cada bloco é de 17
    extension_template2 = '-1004000'
    
    txts = []
    for sample in nsamples:
        rows = []
        for part in nparts:
            name_template = 'predicted_queries_topk_sample0' if sample>9 else 'predicted_queries_topk_sample00'
            extension_template1 = '.txt0' if part>9 else '.txt00'
            
            file_name = f"{name_template}{str(sample)}{extension_template1}{str(part)}{extension_template2}"
            path = os.path.join(folder,file_name)
            txt_reader = open(path,mode = "r",encoding = encoding,errors='ignore')
            for i,row in enumerate(txt_reader):
                rows.append([row.replace("\n","")])

        txts.append(rows)
    
    return txts

In [5]:
collections = load_text_tsv(collections_path)
print("Numero de linhas em collections: ",len(collections))
collections[:2]

Numero de linhas em collections:  8841823


[['0',
  'The presence of communication amid scientific minds was equally important to the success of the Manhattan Project as scientific intellect was. The only cloud hanging over the impressive achievement of the atomic researchers and engineers is what their success truly meant; hundreds of thousands of innocent lives obliterated.'],
 ['1',
  'The Manhattan Project and its atomic bomb helped bring an end to World War II. Its legacy of peaceful uses of atomic energy continues to have an impact on history and science.']]

In [6]:
qrels_train = load_text_tsv(qrels_train_path)
print("Numero de linhas em qrels_train: ",len(qrels_train))
qrels_train[:2]

Numero de linhas em qrels_train:  532761


[['1185869', '0', '0', '1'], ['1185868', '0', '16', '1']]

In [7]:
qrels_dev = load_text_tsv(qrels_dev_path)
print("Numero de linhas em qrels_dev: ",len(qrels_dev))
qrels_dev[:2]

Numero de linhas em qrels_dev:  59273


[['1102432', '0', '2026790', '1'], ['1102431', '0', '7066866', '1']]

In [8]:
queries_train = load_text_tsv(queries_train_path)
print("Numero de linhas em queries_train: ",len(queries_train))
queries_train[:2]

Numero de linhas em queries_train:  808731


[['121352', 'define extreme'],
 ['634306', 'what does chattel mean on credit history']]

In [9]:
queries_dev = load_text_tsv(queries_dev_path)
print("Numero de linhas em queries_dev: ",len(queries_dev))
queries_dev[:2]

Numero de linhas em queries_dev:  101093


[['1048578', 'cost of endless pools/swim spa'], ['1048579', 'what is pcnt']]

In [10]:
queries_eval = load_text_tsv(queries_eval_path)
print("Numero de linhas em queries_eval: ",len(queries_eval))
queries_eval[:2]

Numero de linhas em queries_eval:  101092


[['786436', 'what is prescribed to treat thyroid storm'],
 ['9',
  ' Refer to the data. Diminishing returns begin to occur with the hiring of the _________ unit of labor ']]

In [12]:
topk_queries = load_txts_topk(queries_topk_folder)

In [13]:
print('Numero de queries geradas: ',len(topk_queries[0]))
topk_queries[0][:2]

Numero de queries geradas:  8841823


['what was important to the success of the manhattan project',
 'what were a major contributions to the manhattan effort']

In [3]:
def fix_encode_write(text, tsv_writer):
    try:
        final_text = text.strip() + '\n'
        tsv_writer.writerow(ftfy.fix_encoding(final_text))
    except Exception as e:
        print(f"Encoding fix error? {e}\nTrying to save raw text")
        tsv_writer.writerow(final_text)

In [2]:
import unicodedata
def strip_accents(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)if unicodedata.category(c) != 'Mn')

In [ ]:
encoding="cp1252"
with open('doc2query_onek.tsv', 'w',newline='') as tsv_file:
    tsv_writer = csv.writer(tsv_file, quoting=csv.QUOTE_NONNUMERIC, delimiter='\t', quotechar='"')
#     tsv_writer = csv.writer(tsv_file, delimiter='\t',quotechar='"', lineterminator='\n')

#     for row in topk_queries[0]:
    tsv_writer.writerow(topk_queries[0])
    
#         except:
#             try:
#                 row = re.sub(r'[^\w]', ' ', row)
#                 tsv_writer.writerow(ftfy.fix_encoding(row))
#             except:
#                 row = strip_accents(row)
#                 tsv_writer.writerow(ftfy.fix_encoding(row))

## Unificação das bases

In [62]:
df_collections = pd.DataFrame(collections,columns=['ID_PASSAGE','POSITIVE_PASSAGE'])
df_collections.head()

,ID_PASSAGE,POSITIVE_PASSAGE
0,0,The presence of communication amid scientific ...
1,1,The Manhattan Project and its atomic bomb help...
2,2,Essay on The Manhattan Project - The Manhattan...
3,3,The Manhattan Project was the name for a proje...
4,4,versions of each volume as well as complementa...


In [63]:
df_collections.insert(2, "ARTIFICIAL_QUERY",topk_queries[0])
df_collections.head()

,ID_PASSAGE,POSITIVE_PASSAGE,ARTIFICIAL_QUERY
0,0,The presence of communication amid scientific ...,what was important to the success of the manha...
1,1,The Manhattan Project and its atomic bomb help...,what were a major contributions to the manhatt...
2,2,Essay on The Manhattan Project - The Manhattan...,what is manhattan project
3,3,The Manhattan Project was the name for a proje...,who led the manhattan project
4,4,versions of each volume as well as complementa...,what is manhattan project


In [60]:
df_qrels_train = pd.DataFrame(qrels_train,columns=['ID_QUERY','0','ID_PASSAGE','1'])
df_qrels_train = df_qrels_train.drop(columns=['0', '1'])
df_qrels_train.head()

,ID_QUERY,0,ID_PASSAGE,1
0,1185869,0,0,1
1,1185868,0,16,1
2,597651,0,49,1
3,403613,0,60,1
4,1183785,0,389,1


In [61]:
df_queries_train = pd.DataFrame(queries_train,columns=['ID_QUERY','ORIGINAL_QUERY'])
df_queries_train.head()

,ID_QUERY,ORIGINAL_QUERY
0,121352,define extreme
1,634306,what does chattel mean on credit history
2,920825,what was the great leap forward brainly
3,510633,tattoo fixers how much does it cost
4,737889,what is decentralization process.


In [64]:
df1 = pd.merge(df_collections, df_qrels_train, on='ID_PASSAGE',how='inner')
df1.head()

,ID_PASSAGE,POSITIVE_PASSAGE,ARTIFICIAL_QUERY,ID_QUERY,0,1
0,0,The presence of communication amid scientific ...,what was important to the success of the manha...,1185869,0,1
1,1,The Manhattan Project and its atomic bomb help...,what were a major contributions to the manhatt...,NaN,NaN,NaN
2,2,Essay on The Manhattan Project - The Manhattan...,what is manhattan project,NaN,NaN,NaN
3,3,The Manhattan Project was the name for a proje...,who led the manhattan project,NaN,NaN,NaN
4,4,versions of each volume as well as complementa...,what is manhattan project,NaN,NaN,NaN


In [65]:
df_final_train = pd.merge(df1, df_queries_train, on='ID_QUERY',how='inner')
df_final_train.head()

,ID_PASSAGE,POSITIVE_PASSAGE,ARTIFICIAL_QUERY,ID_QUERY,0,1,ORIGINAL_QUERY
0,0,The presence of communication amid scientific ...,what was important to the success of the manha...,1185869,0,1,)what was the immediate impact of the success ...
1,1,The Manhattan Project and its atomic bomb help...,what were a major contributions to the manhatt...,NaN,NaN,NaN,NaN
2,2,Essay on The Manhattan Project - The Manhattan...,what is manhattan project,NaN,NaN,NaN,NaN
3,3,The Manhattan Project was the name for a proje...,who led the manhattan project,NaN,NaN,NaN,NaN
4,4,versions of each volume as well as complementa...,what is manhattan project,NaN,NaN,NaN,NaN


In [ ]:
compression_opts = dict(method='zip', archive_name='train_quadruples.csv') 
df_final_train.to_csv('train_quadruples.zip', index=False,compression=compression_opts)

In [54]:
%%time
new_table = [['PASSAGE','POSITIVE_QUERY','ORIGINAL_QUERY']]
for passage,artificial_query in zip(collections,topk_queries[0]):
    new_table_line = [passage[1],artificial_query]
    
    for query in queries_train: 
        for qrel in qrels_train:
            if ((qrel[0] == passage[0]) and (qrel[2] == query[0])):
                new_table_line.append(queries_train[1])
                break


    new_table.append(new_table_line) 
    if i ==10:
        break

KeyboardInterrupt: 

In [55]:
new_table

[['PASSAGE', 'POSITIVE_QUERY', 'ORIGINAL_QUERY']]

In [ ]:
for row in new_table:
    with open("output.csv", "wb") as f:
        writer = csv.writer(f)
        writer.writerows(row)

## Unificação de uma vez só

In [ ]:
def create_table(step='train',topk):
    collections_path = 'C:\\Users\\Matheus\\Desktop\\Pós Graduação\\NLP\\Projeto\\Datasets\\MS MARCO 2020\\collections\\collection.tsv'
    qrels_train_path = 'C:\\Users\\Matheus\\Desktop\\Pós Graduação\\NLP\\Projeto\\Datasets\\MS MARCO 2020\\qrels\\qrels_train.tsv'
    qrels_dev_path = 'C:\\Users\\Matheus\\Desktop\\Pós Graduação\\NLP\\Projeto\\Datasets\\MS MARCO 2020\\qrels\\qrels_dev.tsv'
    queries_train_path = 'C:\\Users\\Matheus\\Desktop\\Pós Graduação\\NLP\\Projeto\\Datasets\\MS MARCO 2020\\queries\\queries_train.tsv'
    queries_dev_path = 'C:\\Users\\Matheus\\Desktop\\Pós Graduação\\NLP\\Projeto\\Datasets\\MS MARCO 2020\\queries\\queries_dev.tsv'
    queries_eval_path = 'C:\\Users\\Matheus\\Desktop\\Pós Graduação\\NLP\\Projeto\\Datasets\\MS MARCO 2020\\queries\\queries_eval.tsv'

    
    if step == 'train':
        qrels_path = qrels_train_path
        queries_path = queries_train_path
    else if step == 'dev':
        qrels_path = qrels_dev_path
        queries_path = queries_dev_path
        
     
    with open(complete_path, encoding=encoding, mode = 'r', errors='ignore') as collections_csv_file:
        with open(queries_path, encoding=encoding, mode = 'r', errors='ignore') as queries_csv_file:
            with open(qrels_path, encoding=encoding, mode = 'r', errors='ignore') as qrels_csv_file:
                with open("output.csv", "w") as f:
        
                    collections_reader = csv.reader(collections_csv_file, delimiter='\t')
                    queries_reader = csv.reader(queries_csv_file, delimiter='\t')
                    qrels_reader = csv.reader(collections_csv_file, delimiter='\t')
                    
                    
                    new_table = [['PASSAGE','POSITIVE_QUERY','ORIGINAL_QUERY']]
                    
                    for passage,artificial_query in zip(collections,topk_queries[0]):
                        new_table_line = [passage[1],artificial_query]
                        for qrel in qrels_train:
                            if ((qrel[0] == passage[0]) and (qrel[2] == queries_train[0])):
                                new_table_line.append(queries_train[1])

                        new_table.append(new_table_line) 
                    
                    
                    for row in csv_reader:
                        rows.append(fix_text(row.replace("\n","")))
    
    
    
    
    
    rows = []
    for row in csv_reader:
        rows.append(fix_text(row.replace("\n","")))
    return rows